# Process from webcam

In [19]:
import cv2
from deepface import DeepFace
import os
import numpy as np

# Inicializar la cámara
vid = cv2.VideoCapture(0)

# Fuente para el texto
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (255, 0, 0) 
thickness = 2 
pos = (50, 50) 

# Directorio de emojis
emoji_dir = "./images/emojis"

while True:
    # Capturar fotograma
    ret, frame = vid.read()

    if ret:
        # Analizar la imagen
        obj = DeepFace.analyze(img_path=frame, enforce_detection=False, actions=['gender', 'emotion', 'race', 'age'])
        gender = obj[0]['dominant_gender']
        age = obj[0]['age']
        # Obtener la emoción dominante
        dominant_emotion = obj[0]['dominant_emotion']

        # Obtener el nombre del archivo del emoji correspondiente
        emoji_filename = f"{dominant_emotion.lower()}.png"

        # Construir la ruta completa del emoji
        emoji_path = os.path.join(emoji_dir, emoji_filename)

        # Cargar el emoji
        emoji = cv2.imread(emoji_path, cv2.IMREAD_UNCHANGED)

        # Redimensionar el emoji para que coincida con el tamaño de la región de texto
        emoji = cv2.resize(emoji, (100, 100))

        # Verificar si el emoji tiene un canal de transparencia
        if emoji.shape[2] == 4:
            # Extraer solo los canales RGB del emoji
            emoji_rgb = emoji[:, :, :3]

            # Obtener la máscara de transparencia
            emoji_mask = emoji[:, :, 3] / 255.0

            # Superponer el emoji en el fotograma utilizando la máscara de transparencia
            frame[50:150, 50:150] = (1.0 - emoji_mask[:, :, None]) * frame[50:150, 50:150] + emoji_mask[:, :, None] * emoji_rgb
        else:
            # Si no hay canal de transparencia, simplemente superponer el emoji sin cambios
            frame[50:150, 50:150] = emoji

        # Mostrar el texto con la información de género y raza
        image = cv2.putText(frame, f"{obj[0]['dominant_gender']} {obj[0]['dominant_race']} {obj[0]['age']}  ", pos, font, fontScale, color, thickness, cv2.LINE_AA)

        if age > 30 and gender == "Man":
            # Cargar el emoji
            emoji = cv2.imread('./images/emojis/old_man.png', cv2.IMREAD_UNCHANGED)

        elif age > 30 and gender == "Woman":
            emoji = cv2.imread('./images/emojis/old_woman.png', cv2.IMREAD_UNCHANGED)
            
        # Redimensionar el emoji para que coincida con el tamaño de la región de texto
        emoji = cv2.resize(emoji, (100, 100))

        # Verificar si el emoji tiene un canal de transparencia
        if emoji.shape[2] == 4:
            # Extraer solo los canales RGB del emoji
            emoji_rgb = emoji[:, :, :3]

            # Obtener la máscara de transparencia
            emoji_mask = emoji[:, :, 3] / 255.0

            # Superponer el emoji en el fotograma utilizando la máscara de transparencia
            frame[50:150, 120:220] = (1.0 - emoji_mask[:, :, None]) * frame[50:150, 120:220] + emoji_mask[:, :, None] * emoji_rgb
        else:
            # Si no hay canal de transparencia, simplemente superponer el emoji sin cambios
            frame[50:150, 120:220] = emoji
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    
        # Mostrar el fotograma
        cv2.imshow('Vid', frame)

    # Detener el programa cuando se presiona ESC
    if cv2.waitKey(20) == 27:
        break

# Liberar la cámara
vid.release()
# Cerrar ventanas
cv2.destroyAllWindows()

Action: age: 100%|██████████| 4/4 [00:00<00:00,  4.72it/s]    
